In [1]:
df = spark.sql("SELECT * FROM news_lake.news_table")
display(df)

StatementMeta(, 8d8e0ddb-6c02-4992-8ea9-5fba336bf9b4, 3, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 7572b063-aa6a-4c83-bd3c-51455136f30a)

In [2]:
import synapse.ml.core
from synapse.ml.services import AnalyzeText

model = (
    AnalyzeText()
    .setKind("SentimentAnalysis")
    .setTextCol("description")
    .setOutputCol("response")
    .setErrorCol("error")
)

StatementMeta(, 8d8e0ddb-6c02-4992-8ea9-5fba336bf9b4, 4, Finished, Available, Finished)

In [4]:
result = model.transform(df)

display(result)

StatementMeta(, 8d8e0ddb-6c02-4992-8ea9-5fba336bf9b4, 6, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 74dc9e03-32bf-4a7c-bdb8-ba0121e731b6)

In [14]:
from pyspark.sql.functions import col, when

sentiment_df = result.withColumn("sentiment", col("response.documents.sentiment")) \
                     .withColumn("final_score",
                                  when(col("sentiment") == "positive", col("response.documents.confidenceScores.positive"))
                                 .when(col("sentiment") == "neutral", col("response.documents.confidenceScores.neutral"))
                                 .when(col("sentiment") == "negative", col("response.documents.confidenceScores.negative"))
                                 .otherwise(None))


display(sentiment_df)

StatementMeta(, 8d8e0ddb-6c02-4992-8ea9-5fba336bf9b4, 16, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 7d3bd674-eb59-4049-8f32-8d2162493aed)

In [16]:
from pyspark.sql.utils import AnalysisException

try:

    table_name = 'news_lake.sentiment_analysis_table'

    sentiment_df.write.format('delta').saveAsTable(table_name)

except:

    print('Table exists! Updating...')

    sentiment_df.createOrReplaceTempView('vw_sentiment_df')

    spark.sql(f""" MERGE INTO {table_name} target_table
                   USING vw_sentiment_df source_view

                   ON source_view.url = target_table.url

                   WHEN MATCHED AND

                   source_view.title <> target_table.title OR
                   source_view.description <> target_table.description OR
                   source_view.image_url <> target_table.image_url OR
                   source_view.source <> target_table.source OR
                   source_view.categories <> target_table.categories OR
                   source_view.keywords <> target_table.keywords OR
                   source_view.published_at <> target_table.published_at

                   THEN UPDATE SET *

                   WHEN NOT MATCHED THEN INSERT *
    
                    """)

StatementMeta(, 8d8e0ddb-6c02-4992-8ea9-5fba336bf9b4, 18, Finished, Available, Finished)

Table exists! Updating...
